**نصب کتابخونه های مورد نیاز**

In [ ]:
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!python rapidsai-csp-utils/colab/pip-install.py --quiet

fatal: destination path 'rapidsai-csp-utils' already exists and is not an empty directory.
Installing RAPIDS Stable 25.04
Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com

        ***********************************************************************
        The pip install of RAPIDS is complete.

        Please do not run any further installation from the conda based installation methods, as they may cause issues!

        Please ensure that you're pulling from the git repo to remain updated with the latest working install scripts.

        Troubleshooting:
            - If there is an installation failure, please check back on RAPIDSAI owned templates/notebooks to see how to update your personal files.
            - If an installation failure persists when using the latest script, please make an issue on https://github.com/rapidsai-community/rapidsai-csp-utils
        ***********************************************************************
        


اتصال به اکانت گوگل برای دسترسی به گوگل درایو

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


کپی دیتا ست مورد نظر از گوگل درایو به پوشه کانتنت

In [ ]:
!cp '/content/drive/MyDrive/Colab_Notebooks/TrafficEvents_Aug16_Dec20_Publish.tar.gz' /content/

خارج کردن فایل از حالت فشرده

In [ ]:
!tar -xvzf TrafficEvents_Aug16_Dec20_Publish.tar.gz -C /content/

TrafficEvents_Aug16_Dec20_Publish.csv


ایمپورت کتابخونه های cudf و dask برای کار با دیتاست بزرگ بر روی gpu

In [ ]:
import cudf
import cuml
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import pandas as pd
import dask.dataframe as dd

استفاده از کتابخونه dask برای خواندن از فایل

In [ ]:
ddf = dd.read_csv('/content/TrafficEvents_Aug16_Dec20_Publish.csv')
print(ddf.columns)

Index(['EventId', 'Type', 'Severity', 'TMC', 'Description', 'StartTime(UTC)',
       'EndTime(UTC)', 'TimeZone', 'LocationLat', 'LocationLng',
       'Distance(mi)', 'AirportCode', 'Number', 'Street', 'Side', 'City',
       'County', 'State', 'ZipCode'],
      dtype='object')


استفاده از کتابخونه cudf برای خواندن ستونهای مورد نیاز از فایل

In [ ]:
df = cudf.read_csv(
    '/content/TrafficEvents_Aug16_Dec20_Publish.csv',
    dtype={'City': 'category', 'LocationLat': 'float32', 'LocationLng': 'float32'},
    usecols=['EventId','City', 'LocationLat','LocationLng']
)

# محاسبه آمار برای هر شهر
stats = df.groupby('City').agg(
    meanLat=('LocationLat', 'mean'),
    stdLat=('LocationLat', 'std'),
    meanLng=('LocationLng', 'mean'),
    stdLng=('LocationLng', 'std')
).reset_index()

# پر کردن NaN
stats['stdLat'] = stats['stdLat'].fillna(1)
stats['stdLng'] = stats['stdLng'].fillna(1)

# merge و محاسبه
df = df.merge(stats, on='City', how='left')
df['distanceFromMeanLat'] = (df['LocationLat'] - df['meanLat']) / df['stdLat']
df['distanceFromMeanLng'] = (df['LocationLng'] - df['meanLng']) / df['stdLng']

print(df)

In [ ]:
filtered_df = df[(df["distanceFromMeanLat"] > 1.0) & (df["distanceFromMeanLng"] > 1.0)]
print(filtered_df)

             EventId  LocationLat  LocationLng             City    meanLat  \
72           T-41208    37.442997  -121.890564         Milpitas  37.430229   
359          T-43191    36.848618  -121.771164     Moss Landing  36.824455   
456          T-39160    38.406509  -121.910011        Vacaville  38.374549   
500          T-41572    38.207676  -122.156708  American Canyon  38.187835   
654          T-46462    37.242973  -121.964859        Los Gatos  37.195001   
...              ...          ...          ...              ...        ...   
31355418  T-31957480    33.870693  -117.742424          Anaheim  33.822793   
31355426  T-31957488    33.870941  -117.740051          Anaheim  33.822793   
31355477  T-31957043    34.108128  -118.185585      Los Angeles  34.003707   
31355503  T-31957133    34.073830  -118.233261      Los Angeles  34.003707   
31355566  T-31957019    33.989296  -118.219345  Huntington Park  33.979496   

            stdLat     meanLng    stdLng  distanceFromMeanLat  

In [ ]:
# محاسبه تعداد outlierها
outlier_count = (
    filtered_df.groupby("City")
               .agg(outlierCount=("EventId", "count"))
               .reset_index()
)
print(outlier_count)

In [ ]:
eventIds=filtered_df["EventId"].to_arrow().to_pylist()
print(eventIds)


In [ ]:
print(len(eventIds))

867500
